In [40]:
import pandas as pd
#%matplotlib notebook
#import seaborn as sns

df_ALL = pd.read_sas("./HN16_ALL.sas7bdat", format = 'sas7bdat', encoding='iso-8859-1')
#df_ALL.head()
df_ffq = pd.read_sas("./hn16_ffq.sas7bdat", format = 'sas7bdat', encoding='iso-8859-1')
#df_ffq.head()

df = pd.merge(df_ALL, df_ffq, how='left', on=[
                                  'mod_d', # 최종DB 수정일
                                  'ID', # 개인 아이디
                                  'ID_fam', # 가구 아이디
                                  'year', # 조사연도
                                  'region', # 17개 시도
                                  'town_t', # 동/읍면 구분
                                  'apt_t', # 아파트 구분, 1.일반, 2.아파트
                                  'psu', # 조사구번호
                                  'sex', # 성별, 1.남자, 2.여자
                                  'age', # 만나이, 80은 80이상 포함(탑코딩)
                                  'age_month', # 만 1~6세의 경우 개월수(월령)
                                  'incm', # 소득 사분위수(개인), 1.하, 2.중하, 3.중상, 4.상
                                  'ho_incm', # 소득 사분위수(가구),  1.하, 2.중하, 3.중상, 4.상
                                  'edu', # 교육수준 재분류 코드, 1.초졸이하, 2.중졸, 3.고졸, 4.대졸이상
                                  'occp', # 직업재분류 및 실업/비경제활동 상태코드
                                  'wt_hs', # 가구조사 가중치
                                  'wt_itvex', # 건강설문-검진조사 가중치
                                  'wt_pft', # 폐기능검사 가중치(만40세이상)
                                  'wt_hm', # 중금속검사 가중치(만10세이상)
                                  'wt_ntr', # 영양조사 가중치
                                  'wt_tot', # 건강설문-검진조사 & 영양조사 가중치
                                  'wt_pfhm', # 폐기능검사 & 중금속검사 가중치(만40세이상)
                                  'wt_pfnt', # 폐기능검사 & 영양조사 가중치(만40세이상)
                                  'wt_hmnt', # 중금속검사 & 영양조사 가중치(만10세이상)
                                  'wt_pfhmnt', # 폐기능검사 & 중금속검사 & 영양조사 가중치(만40세이상)
                                  'kstrata' # 분산추정 층
                                 ])

df_data = df.loc[(18 < df.age), :]

df_data = df_data.loc[(1 < df_data.DI1_pt) | (df_data.DI1_pt < 1), :]

In [41]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import graphviz
from sklearn import tree
from tqdm import tqdm_notebook

def data_clean(input_list, is_cap=False) :
    tmp_list = []
    for d in input_list :
        try :
            float(d)
            if is_cap :
                tmp_list.append([d])
            else :
                tmp_list.append(d)
        except :
            if is_cap :
                tmp_list.append([-1])
            else :
                tmp_list.append(-1)
    return tmp_list

def saveDTR(input_df, x_name, y_name) :
    arrX1 = data_clean(getattr(input_df,x_name).fillna(-1), is_cap=True)
    arrY = data_clean(getattr(input_df,y_name).fillna(-1))
    
    model = DecisionTreeRegressor(
        criterion = 'mse',
        max_depth=2
    )
    model.fit(arrX1,arrY)

#     dot_data = tree.export_graphviz(model,out_file=None,feature_names=[x_name], class_names=[y_name])

#     graph = graphviz.Source(dot_data) 
#     graph.render("%s+%s" % (x_name,y_name))

    # mse값 계산. 정상적으로 만든건지는 모르겠음..
    modelPrediction = model.predict(arrX1)
    mse = mean_squared_error(arrY,modelPrediction)
    # x_name, y_name, mse값을 mse_list에 append
    mse_list.append([x_name, y_name, mse])
    
skip_list = [
    df_data.HE_dbp.name
    , df_data.HE_sbp.name
]

# saveDTR(df_data, df_data.age.name, df_data.HE_dbp.name)
# saveDTR(df_data, df_data.age.name, df_data.HE_sbp.name)

# mse list 담을 변수 준비
mse_list = []

for x_name in tqdm_notebook(df_data.columns):
    if x_name in skip_list : continue
    saveDTR(df_data, x_name, df_data.HE_dbp.name)
    saveDTR(df_data, x_name, df_data.HE_sbp.name)

# mse_list > dataframe으로 변환해서 csv파일로 저장   
df_mse = pd.DataFrame(mse_list)
df_mse.to_csv("mse_list.csv")